In [1]:
!pip -V

pip 24.1.2 from C:\projects\feature-map-experiments\.venv\lib\site-packages\pip (python 3.10)



In [2]:
!pip install git+https://github.com/mhamilton723/FeatUp.git

  Running command git clone --filter=blob:none --quiet https://github.com/mhamilton723/FeatUp.git 'C:\Users\artif\AppData\Local\Temp\pip-req-build-rkovizc7'
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [6 lines of output]
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\artif\AppData\Local\Temp\pip-req-build-rkovizc7\setup.py", line 2, in <module>
          from torch.utils.cpp_extension import BuildExtension, CUDAExtension, CppExtension
      ModuleNotFoundError: No module named 'torch'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above f


  Cloning https://github.com/mhamilton723/FeatUp.git to c:\users\artif\appdata\local\temp\pip-req-build-rkovizc7
  Resolved https://github.com/mhamilton723/FeatUp.git to commit 6b5a6c0e91f75e69194807128dcbc39c3084a30d
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
